In [ ]:
import numpy as np
import torchvision
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from skimage import io
import torch
import torch.nn as nn
from einops import rearrange, repeat
import pandas as pd
import sys
sys.path.insert(0, '../utils/')
from dataset import ChestImage64
import os
import time
import datetime

import matplotlib.pyplot as plt

from torchvision.models import vit_l_16, ViT_L_16_Weights,vit_b_16, ViT_B_16_Weights
from torch.utils.data import random_split, DataLoader, Dataset

In [ ]:
csv_path = "../Data/256pxImages/train_labels_256p_paired.csv"
root_path = '../Data/256pxImages'

pairCSV = pd.read_csv(csv_path)
pairCSV['EncodedLabels'] = ''
print(pairCSV.shape)

for i in range(4, pairCSV.shape[1]-1):
    pairCSV['EncodedLabels'] = pairCSV['EncodedLabels'].astype(str) + pairCSV.iloc[:, i].astype(str) 
    if i < pairCSV.shape[1]-2:
        pairCSV['EncodedLabels'] = pairCSV['EncodedLabels'].astype(str) + "," 

pairCSV.head()

In [ ]:
# test image loading
front_file = pairCSV.iloc[0, 0]
lat_file= pairCSV.iloc[0, 1]
label_test = pairCSV['EncodedLabels'].iloc[0]

test_path_front = os.path.join(root_path, front_file)
test_path_lat = os.path.join(root_path, lat_file)

label_test = [int(x) for x in label_test.split(",")]

print("label_test: ", label_test)

image_front = io.imread(test_path_front)
print(type(image_front))
image_front = torch.tensor(image_front)
print(image_front.size())

image_lat = io.imread(test_path_lat)
print(type(image_lat))
image_lat = torch.tensor(image_lat)
print(image_lat.size())

In [ ]:
# load up the paired dataset
class PairedDataset(Dataset):
    def __init__(self, df, root_dir, label_col, transform = None):
        self.df = df
        self.root_dir = root_dir
        self.label_col = label_col
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        if torch.is_tensor(index):
            index = index.tolist()

        # get the filename of the image
        file_front = self.df.iloc[index, 0]
        file_lat = self.df.iloc[index, 1]
        label = self.df[self.label_col].iloc[index]

        if type(label) == str:
            label = [int(x) for x in label.split(",")]

        # load the image from disk
        front_path = os.path.join(self.root_dir, file_front)
        lat_path = os.path.join(self.root_dir, file_lat)

        img_front = io.imread(front_path)
        img_lat = io.imread(lat_path)

        label = torch.tensor(label)
        label = label.float()

        img_front = torch.tensor(img_front)
        img_front = img_front.resize_((224, 224))
        img_front = repeat(img_front, "h w -> (repeat h) w", repeat = 3)
        img_front = rearrange(img_front, "(c h) w -> 1 c h w", c = 3)
        img_front = img_front.float()

        img_lat = torch.tensor(img_lat)
        img_lat = img_lat.resize_((224, 224))
        img_lat = repeat(img_lat, "h w -> (repeat h) w", repeat = 3)
        img_lat = rearrange(img_lat, "(c h) w -> 1 c h w", c = 3)
        img_lat = img_lat.float()

        img_pair=torch.cat((img_front,img_lat),0)


        # if self.transform:
            # label = self.transform(label)
            # img = self.transform(img)

        # return the image and its filename
        return img_pair, label
    

In [ ]:
#default_transform = ViT_L_16_Weights.IMAGENET1K_SWAG_LINEAR_V1.transforms
#pairDataset = PairedDataset(pairCSV, root_dir=root_path, label_col="EncodedLabels", transform=default_transform)

#print(type(pairDataset))

In [ ]:
# split into test train validate
#train_size = int(0.7 * len(pairDataset))
#val_size = int(0.1 * len(pairDataset))
#test_size = int(0.2 * len(pairDataset))


#train_dataset, val_dataset, test_dataset = random_split(pairDataset, [train_size, val_size, test_size])

#print("Train Length: ", len(train_dataset))
#print("Validation Length: ", len(val_dataset))
#print("Test Length: ", len(test_dataset))

#batchsize = 8

# make three different dataloaders
#train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=batchsize, shuffle=False)
#test_loader = DataLoader(test_dataset,batch_size=batchsize, shuffle=True)


#features, labels = next(iter(train_loader))
#print(features.size())
#print(features.dtype)

#print(features[1, 1, :, :])

#print(labels.size())
#print(datetime.datetime.now().strftime("%H:%M:%S"))

In [ ]:
front_features=features[:, 0, :, :, :]
print(front_features.size())

In [ ]:
lat_features=features[:, 1, :, :, :]
print(lat_features.size())

In [ ]:
class AllAttentionVIT(nn.Module):
    def __init__(self,weights_frontal,weights_lateral,code_size,num_classes,bottleneck=1024,drop_rate=0.1):
        
        super(AllAttentionVIT,self).__init__()

        #Initialize the model
        self.transformer_enc_frontal = vit_b_16(weights = weights_frontal)
        self.transformer_enc_frontal.heads = nn.Sequential(nn.Linear(768, code_size))
        self.transformer_enc_lateral = vit_b_16(weights = weights_lateral)
        self.transformer_enc_lateral.heads = nn.Sequential(nn.Linear(768, code_size))

        self.dropout = nn.Dropout(p = drop_rate)
        self.ff = nn.Linear(code_size*2,num_classes)

    #Expect two images with the same class.
    def forward(self,x):

        #Split up paired images
        x_f=x[:, 0, :, :, :]
        x_l=x[:, 1, :, :, :]
        
        #Encode front and lateral embeddings
        x_f = self.transformer_enc_frontal(x_f)
        x_l = self.transformer_enc_frontal(x_l)

        #Concat embeddings. May want to experiment with convolutions.
        x = torch.cat((x_f,x_l),1)

        #Map concatenated embeddings onto the classes
        x = self.dropout(x)
        x = self.ff(x)

        return x

In [ ]:
#testWeights=ViT_B_16_Weights.DEFAULT
#testModel = AllAttentionVIT(testWeights,testWeights,768,14)

# simple function to determine how many TRAINABILE parameters are in the model
def count_parameters(testModel):
    return sum(p.numel() for p in testModel.parameters() if p.requires_grad)

print(count_parameters(testModel))

In [ ]:
# print out a list of the parameters we are training
for name,param in testModel.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

In [ ]:
# batch accumulation parameter
target_accumulation=32
accum_iter = target_accumulation/batchsize  
print(accum_iter)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def train_multimodal_model(train_dataset, validation_dataset,config, benchmark=0.33):
    # now let's train this thing. 
    
    batchsize = 8
    epochs = 1
    learning_rate = 0.003
    
    # split into test train validate
    train_size = int(0.7 * len(pairDataset))
    val_size = int(0.1 * len(pairDataset))
    test_size = int(0.2 * len(pairDataset))


    train_dataset, val_dataset, test_dataset = random_split(pairDataset, [train_size, val_size, test_size])
    
    default_transform = ViT_L_16_Weights.IMAGENET1K_SWAG_LINEAR_V1.transforms
    pairDataset = PairedDataset(pairCSV, root_dir=root_path, label_col="EncodedLabels", transform=default_transform)
    
    
    train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batchsize, shuffle=False)
    test_loader = DataLoader(test_dataset,batch_size=batchsize, shuffle=True)
    
    features, labels = next(iter(train_loader))
    
    testWeights=ViT_B_16_Weights.DEFAULT
    testModel = AllAttentionVIT(testWeights,testWeights,768,14)

    criterion = nn.BCEWithLogitsLoss()

    # make sure the model is running on the GPU if its available
    testModel.to(device)

    # Observe that all parameters are being optimized
    params_to_update = testModel.parameters()
    optimizer = torch.optim.SGD(params_to_update, lr=learning_rate, momentum=0.9)


    # optimizer = optim.Adam(model500k.parameters(), lr=learning_rate)

    # start a clock 
    print("Training Starting.")
    start_time = time.time()

    # create empty arrays to hold the loss results
    training_losses = []
    validation_losses = []
    for epoch in range(epochs):
        phase = 'train'
        # set the model to training mode
        testModel.train()
        running_loss = 0.0

    # zero the parameter gradients at the very beginning
        optimizer.zero_grad()


        for i, data in enumerate(train_loader):
            inputs, labels = data[0].to(device), data[1].to(device)

            # ugh. This is gross. I should have done this step at the beginning for all of the datasets. 
            # zero the parameter gradients
            optimizer.zero_grad()
            # print("Here's the size of the inputs: ", inputs.size())
            # with torch.set_grad_enabled(phase == 'train'):
                # run the training data through the model
            outputs = testModel(inputs)

            #calculate the loss of the model
            loss = criterion(outputs, labels)

            #Gradient accumulation
            loss = loss / accum_iter
            loss.backward()

            running_loss += loss.item()

            # weights update for each gradient accumulation
            if ((i + 1) % accum_iter == 0) or (i + 1 == len(train_loader)):
                optimizer.step()
                optimizer.zero_grad()

                if ((i + 1) / accum_iter) % 50 == 0:    # record loss and test validation set every 50 gradient accumulations
                    testModel.eval()
                    v_running_loss = 0.0
                    for v, data in enumerate(val_loader):
                        inputs, labels = data[0].to(device), data[1].to(device)
                        v_outputs = testModel(inputs)
                        v_loss = criterion(v_outputs, labels)
                        v_running_loss += v_loss.item()

                    print("Time: ", datetime.datetime.now().strftime("%H:%M:%S"), "\tepoch: ", epoch+1, "batch: ", i+1, "Training loss: ", running_loss, "Validation loss ", v_running_loss)
                    validation_losses.append(v_running_loss)
                    training_losses.append(running_loss)
                    running_loss = 0.0


    end_time = time.time()
    train_time = end_time - start_time
    print("Elapsed Training Time: ", datetime.timedelta(seconds = train_time))
    print('Finished Training')


In [ ]:
result_list = []

#Grid search parameters
#dropout=[0.2]
batch_size=[4]
learn_rate=[1]
epochs=[1]


it=0
#for d in dropout:
for e in epochs:
    for bs in batch_size:
        for lr in learn_rate:

            it +=1 
            #config = {"d1": d,"lr": lr,"batch_size": bs}
            config = {"epochs": e, "lr": lr, "batch_size": bs}
            #config = {"lr": lr,"batch_size": bs}

            print(config)
            print("Train Start. Iteration: ",it)
            t_loss,v_loss, end_Ep=train_model(train_dataset, val_dataset,config)
            #t_loss,v_loss, acc, end_Ep=train_model(train_dataset, val_dataset,config)

            list_row= str(bs) + "," + str(lr) + "," + str(t_loss) + "," + str(v_loss) + "," + str(end_Ep)
            #list_row= str(bs) + "," + str(lr) + "," + str(t_loss) + "," + str(v_loss) + "," + str(end_Ep), + "," + str(acc)

            t_list=[list_row]  
            result_list.append(t_list)


with open('opt_results_multimodal.csv','w') as result_file:
    wr = csv.writer(result_file, dialect='excel')
    wr.writerow(result_list)